<a href="https://colab.research.google.com/github/Mihawk1891/PR-LLM-and-RAG-based-Invoice-Data-Retrieve-Based-on-User/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import google.generativeai as genai
from langchain_community.vectorstores import FAISS



In [ ]:
# Set up Google API key
os.environ["GOOGLE_API_KEY"] = "Enter your API"
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])



In [ ]:
# Load Documents using PyPDF LOader
files={"invoice":["invoice (1).pdf","invoice(2).pdf"]}## Enter relevant path of your files



start=0

for invoice in files["invoice"]:
    print(invoice)
    loader = PyPDFLoader(invoice) # change document Here
    pages = loader.load()
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    db = FAISS.from_documents(pages,embeddings ) # creating Faiss Vector store

    if start==0:
        db1=db
        start+=1
    else:
        db1.merge_from(db)


relevant_invoices = db1.as_retriever()

loader = PyPDFLoader("invoice_structure_and_terms.pdf") # change document Here
pages = loader.load()
db = FAISS.from_documents(pages,embeddings )
relevant_knowledge = db.as_retriever()



invoice (1).pdf
invoice(2).pdf


Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)


In [ ]:
prompti = """
User Query: {user_input}

Relevant Invoice Data:
{relevant_invoices}

Relevant Invoice Knowledge:
{relevant_knowledge}

You are a document analysis assistant. Based on the User Query, the relevant invoice data, and the knowledge about invoice structures and terms, please provide a detailed and accurate response.If you need any clarification or additional information, please ask.
The Answer Should be points and then subpoints. No paragraph until it is required.

Focus solely on the document content to answer the user's question.If there is a user query that Do not reference or utilize any external knowledge or information beyond what is explicitly stated within the document then answer 'Please ask questions on the invoice'
Donot tell what the user asked. If the user is vague just provide the answers and also give suggestive questions.


You have the Chat History below:
"""

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [ ]:
# Create prompt template

def llm_ans(chat_input,history):
    global prompti
    prompt2=f"""{prompti}\n {history}"""
    prompt = ChatPromptTemplate.from_template(prompt2)

    llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)
    chain = (
        {"relevant_invoices": relevant_invoices,"relevant_knowledge":relevant_knowledge ,"user_input": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
        )

    return chain.invoke(chat_input)



In [ ]:
chathistory=""

while True:
    user=input()
    ans=llm_ans(user,chathistory)
    chathistory+=f"HumanMessage:{user}\n"
    chathistory+=f"AIMessage:{ans}\n\n"

    print("input:",user)
    print("Output:",ans,"\n\n")



input: 
Output:  


input: Give me the invoice details
Output:  


input: Give me the invoice details
Output: ## Invoice Details:

* **Invoice 1:**

    * **Invoice Number:** AMD2-3878067
    * **Invoice Date:** 06.12.2023
    * **Order Number:** 404-9193373-2752348
    * **Order Date:** 06.12.2023
    * **Seller:** COCOBLU RETAIL LIMITED 
        * GST Registration No: 24AAJCC8517E1ZR 
        * PAN No: AAJCC8517E
    * **Buyer:** Pravin Misal
    * **Items:**
        * realme Buds 2 Wired in Ear Earphones with Mic (Blue) | B0CJQZ8BQQ ( B0CJQZ8BQQ ) 
            * Quantity: 1
            * Unit Price: ₹507.63
            * Total Amount: ₹599.00
    * **Shipping Charges:** ₹33.90
    * **Total Amount:** ₹599.00

* **Invoice 2:**

    * **Invoice Number:** 36258
    * **Invoice Date:** Mar 06 2012
    * **Seller:** SuperStore
    * **Buyer:** Aaron Bergman
    * **Items:**
        * Global Push Button Manager's Chair, Indigo
            * Quantity: 1
            * Unit Price: $48.71
   

In [ ]:
print(chathistory)